In [1]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import glob
import os
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.structures import BoxMode

from detectron2.config import get_cfg

/opt/conda/lib/python3.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
class_names = ["specularity", "saturation", "artifact", "blur", "contrast", 
               "bubbles", "instrument", "blood"]

# Things to change:
# 1: model_path
# 2: weights => _009999.pth
# 3: SCORE_THRESH_TEST

model_path = "model_cascade_mask_rcnn_R_50_FPN_3x/output_23/"

test_types = ['Detection', 'Detection_sequence', 'Generalization']

for test_type in test_types:

    test_result_type=''

    if test_type == 'Detection':
        test_result_type = 'detection_bbox'
    elif test_type == 'Detection_sequence':
        test_result_type = 'sequence_bbox'
    elif test_type == 'Generalization':
        test_result_type = 'generalization_bbox'

    cfg = get_cfg()

    cfg.merge_from_file(model_path+'config.yaml')
    cfg.MODEL.WEIGHTS = os.path.join(model_path, "model_0059999.pth")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3   
    cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.3
    
    cfg.DATASETS.TEST = ("ead_validation_1",)

    predictor = DefaultPredictor(cfg)

    VALIDATION_PATH = r'/home/ws2080/Desktop/data/EAD_TEST_FINAL/'+test_type+'/'
    TEST_IMAGE_PATHS = glob.glob(VALIDATION_PATH+"*.jpg")

    for path in TEST_IMAGE_PATHS:
        im = cv2.imread(path)
        outputs = predictor(im)

        txt_home_folder = r'/home/ws2080/Desktop/data/EAD_TEST_FINAL_predictions/'+test_result_type+'/'

        txt_file_name = path.split("/")[7]
        txt_file_name = txt_file_name.replace(".jpg", ".txt")
        txt_file_name = txt_home_folder+txt_file_name    

        total_detection = len(outputs["instances"])

        temp_detection_list = []
        detections = outputs["instances"]

        for i in range(total_detection):
            temp_detection = class_names[int(detections.pred_classes[i])]+" "+ str(float(detections.scores[i]))+" "+str(float(detections.pred_boxes.tensor[i,0]))+" "+ str(float(detections.pred_boxes.tensor[i,1])) +" "+str(float(detections.pred_boxes.tensor[i,2]))+" "+ str(float(detections.pred_boxes.tensor[i,3]))
            temp_detection_list.append(temp_detection)

        with open(txt_file_name, 'w') as f:
                for item in temp_detection_list:
                    f.write("%s\n" % item)                                                                           

    print(test_result_type, " created")   

detection_bbox  created
sequence_bbox  created
generalization_bbox  created
